# AST 4930 Module 7

### Deep Neural Networks

In [ ]:
from sklearn.datasets import load_iris
import numpy as np
import matplotlib.pyplot as plt 

iris = load_iris()
X = iris.data[:, (2, 3)]  # petal length, petal width
y = iris.target

plt.scatter(X[:,0],X[:,1],c=y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
X_train.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

X_train_scaled = StandardScaler().fit(X_train).transform(X_train)
X_test_scaled = StandardScaler().fit(X_train).transform(X_test)

### Let's build our first neural network. We will use sklearn's MLPClassifier

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
help(MLPClassifier)

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(100,100),activation="relu",
                      max_iter=1000,random_state=1,verbose=1)

model.fit(X_train_scaled, y_train)

y_pred=model.predict(X_test_scaled)
print(model.score(X_test_scaled, y_test))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, model.predict(X_test_scaled), labels=model.classes_)

fig=ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
fig.plot()
plt.show()

In [ ]:
model.predict_proba(X_test_scaled)

In [ ]:
# These are weights.

print(model.coefs_[0].shape, model.coefs_[1].shape, model.coefs_[2].shape)

In [ ]:
model.coefs_[0]

In [ ]:
# These are weights for the bias term.

print(model.intercepts_[0].shape, model.intercepts_[1].shape, model.intercepts_[2].shape)

In [ ]:
model.intercepts_[0]

In [ ]:
plt.plot(model.loss_curve_)
plt.xlabel('Iterations')
plt.ylabel('Training Loss')

### Hyperparameter Optimization

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('sc', StandardScaler()),
    ('mlp', MLPClassifier())
])

param_grid = {
    'mlp__hidden_layer_sizes': [(75,50,25), (150,100,50), (300,200,100)],
    'mlp__max_iter': [1000, 1500, 2000],
    'mlp__solver': ['sgd', 'adam'],
}

grid_search = GridSearchCV(pipe, param_grid, return_train_score=True, cv=5, n_jobs=1, verbose=3)
grid_search.fit(X_train, y_train)

In [ ]:
print("Best parameters: {}".format(grid_search.best_params_))
print("Best model: {}".format(grid_search.best_estimator_))
print("Test score: {:.2f}".format(grid_search.score(X_test, y_test)))

### Save and load a model.

In [ ]:
import pickle

# save
with open('iris_model.pkl','wb') as f:
    pickle.dump(grid_search.best_estimator_,f)

# load
with open('iris_model.pkl', 'rb') as f:
    model_best = pickle.load(f)

In [ ]:
model_best.get_params()

In [ ]:
model_best.predict(X_test)

In [ ]:
model_best.score(X_test, y_test)

### Instead of sklearn's MLP classifier, we can use Keras implemented onto Tensorflow. You will have to choose a Tensorflow kernel on hipergator.


Keras: https://keras.io/

Tensorflow: https://www.tensorflow.org/

In [ ]:
import tensorflow.keras as keras

# Initialize a model.
model = keras.models.Sequential()

# Add the input layer and specifying its shape.
model.add(keras.layers.Input(shape=X_train.shape[1]))

# Add the first hidden layer with 100 neurons and the ReLU activation function.
model.add(keras.layers.Dense(100, activation='relu'))

# Add the second hidden layer with 100 neurons and the ReLU activation function.
model.add(keras.layers.Dense(100, activation='relu'))

# Add the output layer
# softmax normalizes the output to a probability distribution
model.add(keras.layers.Dense(np.unique(y).shape[0], activation='softmax'))

In [ ]:
y

In [ ]:
np.unique(y).shape[0]

### Let's check the "architecture" of our neural network. Note that the input layer does not show up here.

In [ ]:
model.summary()

### We can use the following syntax instead.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Input(shape=X_train.shape[1]),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(100, activation='relu',),
    keras.layers.Dense(np.unique(y).shape[0], activation='softmax')
])

### Let's check the "architecture" of our neural network. Note that the input layer does not show up here.

In [ ]:
model.summary()

### We can also have a look at the weights.

In [ ]:
model.weights

### Now, we need to "compile" the model before we fit to the data.

Some useful links

Loss functions: https://keras.io/api/losses/ 

Optimizers: https://keras.io/api/optimizers/ 

Metrics: https://keras.io/api/metrics/

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics='accuracy')

### We are ready to train the neural network. Let's start training.

In [ ]:
# history will record the loss, accuracy, etc. 
# validation_split will split the training dataset into training + validation datasets. 
history = model.fit(X_train_scaled, y_train, epochs=1000, validation_split=0.2)

3/3 [==============================] - 0s 21ms/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 1.1911 - val_accuracy: 0.9130
Epoch 788/1000
3/3 [==============================] - 0s 21ms/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 1.1756 - val_accuracy: 0.9130
Epoch 789/1000
3/3 [==============================] - 0s 21ms/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 1.1700 - val_accuracy: 0.9130
Epoch 790/1000
3/3 [==============================] - 0s 21ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 1.1892 - val_accuracy: 0.9130
Epoch 791/1000
3/3 [==============================] - 0s 22ms/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 1.2018 - val_accuracy: 0.9130
Epoch 792/1000
3/3 [==============================] - 0s 21ms/step - loss: 0.0042 - accuracy: 1.0000 - val_loss: 1.2161 - val_accuracy: 0.9130
Epoch 793/1000
3/3 [==============================] - 0s 20ms/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 1.2039 - val_accuracy: 0.9130
Epoch 794/1000

3/3 [==============================] - 0s 21ms/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 1.3018 - val_accuracy: 0.9130
Epoch 902/1000
3/3 [==============================] - 0s 21ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 1.3093 - val_accuracy: 0.9130
Epoch 903/1000
3/3 [==============================] - 0s 21ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 1.3189 - val_accuracy: 0.9130
Epoch 904/1000
3/3 [==============================] - 0s 22ms/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 1.3084 - val_accuracy: 0.9130
Epoch 905/1000
3/3 [==============================] - 0s 22ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 1.3012 - val_accuracy: 0.9130
Epoch 906/1000
3/3 [==============================] - 0s 22ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 1.2958 - val_accuracy: 0.9130
Epoch 907/1000
3/3 [==============================] - 0s 22ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 1.3132 - val_accuracy: 0.9130
Epoch 908/1000

### Note that in each epoch, there are 3 mini batches.

In [ ]:
(X_train.shape[0]*0.8)/32

### Let's evlauate the model using the test dataset.

In [ ]:
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

### Let's check how loss and validation loss evolved over epoch.

In [ ]:
plt.plot(history.epoch, history.history['loss'],'.-', label='loss')
plt.plot(history.epoch, history.history['val_loss'],'.-', label='validation loss')
plt.xlabel('epoch')
plt.ylabel('loss, validation loss')
plt.legend()

### Let's check how accuracy and validation accuracy evolved over epoch.

In [ ]:
plt.plot(history.epoch, history.history['accuracy'],'.-', label='accuracy')
plt.plot(history.epoch, history.history['val_accuracy'],'.-', label='validation accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy, validation accuracy')
plt.legend()

### Early stopping

In [ ]:
# The default monitor is "val_loss"
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
checkpoint_cb = keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Input(shape=X_train.shape[1]),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(100, activation='relu',),
    keras.layers.Dense(np.unique(y).shape[0], activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics='accuracy')

In [ ]:
history = model.fit(X_train, y_train, epochs=1000,
                    validation_split=0.2,
                    callbacks=[checkpoint_cb, early_stopping_cb])


In [ ]:
plt.plot(history.epoch, history.history['loss'],'.-', label='loss')
plt.plot(history.epoch, history.history['val_loss'],'.-', label='validation loss')
plt.xlabel('epoch')
plt.ylabel('loss, validation loss')
plt.legend()

In [ ]:
plt.plot(history.epoch, history.history['accuracy'],'.-', label='accuracy')
plt.plot(history.epoch, history.history['val_accuracy'],'.-', label='validation accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy, validation accuracy')
plt.legend()

### Let's check out weights and biases.

In [ ]:
model.layers

In [ ]:
hidden1 = model.layers[0]

In [ ]:
hidden1.name

### Let's put weights and biases of the first hidden layer in w1 and b1.

In [ ]:
w1, b1 = hidden1.get_weights()

In [ ]:
w1

In [ ]:
w1.shape

In [ ]:
b1

In [ ]:
b1.shape

### We can save the model and then load it as needed.

In [ ]:
model.save("iris_model_final.h5")

In [ ]:
model_new = keras.models.load_model("iris_model_final.h5")

In [ ]:
model_new.summary()

### Let's try a larger dataset = the MNIST dataset.

In [ ]:
import tensorflow.keras as keras

(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

: 

### Scale the images to 0 - 1.

In [ ]:
X_train = X_train / 255.
X_test = X_test / 255.

### For now, let's use 10% of the data.

In [ ]:
X_train = X_train[::10]
X_test = X_test[::10]

y_train = y_train[::10]
y_test = y_test[::10]

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
import matplotlib.pyplot as plt 

plt.imshow(X_train[0], cmap="binary")
plt.axis('off')
plt.show()

### TODO: Let's build a NN with two hidden layers, each having 1000 neurons.

### TODO: Print out the model summary and make sure things are correctly set up.

### Compile the model. Let's first try Stochastic Gradient Descent with learning rate of 1.0e-3.

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

### TODO: Train the model using 100 epochs and 20 % of the data in the validation dataset.

### TODO: Evaluate the model using the test dataset.

### Let's check some of the results.

In [ ]:
n = 0

# Display the image.
plt.imshow(X_test[n])

# Model prediction: probability for each class.
y_prob = model.predict(np.expand_dims(X_test[n], axis=0))
y_prob.round(3)

### The evaluation score is 88%, which is decent but not very impressive. Why? Can we improve the performance?

### Let's check how loss and accuracy have evolved over the training.

In [ ]:
plt.plot(history.epoch, history.history['loss'],'.-', label='loss')
plt.plot(history.epoch, history.history['val_loss'],'.-', label='validation loss')
plt.xlabel('epoch')
plt.ylabel('loss, validation loss')
plt.legend()

In [ ]:
plt.plot(history.epoch, history.history['accuracy'],'.-', label='accuracy')
plt.plot(history.epoch, history.history['val_accuracy'],'.-', label='validation accuracy')
plt.ylim(0.7,1)
plt.xlabel('epoch')
plt.ylabel('accuracy, validation accuracy')

### It seems like the model is still underfitting. We can increase the number of epochs or learning rate, but we can try to use the "adam" optimizer which will monitor and adjust the learning rate as the training proceeds.

In [ ]:
model2 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(1000, activation="relu"),
    keras.layers.Dense(1000, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model2.summary()

In [ ]:
model2.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              metrics=["accuracy"])

In [ ]:
history2 = model2.fit(X_train, y_train, epochs=100, validation_split=0.2)

In [ ]:
loss, accuracy = model2.evaluate(X_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

### Let's check how loss and accuracy have evolved over the training.

In [ ]:
plt.plot(history2.epoch, history2.history['loss'],'.-', label='loss')
plt.plot(history2.epoch, history2.history['val_loss'],'.-', label='validation loss')
plt.xlabel('epoch')
plt.ylabel('loss, validation loss')
plt.legend()

In [ ]:
plt.plot(history2.epoch, history2.history['accuracy'],'.-', label='accuracy')
plt.plot(history2.epoch, history2.history['val_accuracy'],'.-', label='validation accuracy')
plt.ylim(0.9,1.01)
plt.xlabel('epoch')
plt.ylabel('accuracy, validation accuracy')
plt.legend()